In [2]:
# https://plot.ly/python/subplots/
import plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs

py.offline.init_notebook_mode(connected=True)

import numpy as np
from sklearn.metrics import mean_squared_error

def print_mse(got, predicted, prefix=''):
    mse = mean_squared_error(got, predicted)
    if isinstance(mse, np.complex):
        print('%s Mean Squared Error: (%.2g, %.2g)' % (prefix, mse.real, mse.imag))
    else:
        print('%s Mean Squared Error: %.2g' % (prefix, mse))
        



In [3]:
# фильтр скользящего среднего
def create_filter_median(kernel_size):
    if kernel_size % 2 == 0:
        raise ValueError("kernel_size should be odd")
    return [1 / kernel_size for i in range(kernel_size)]
    
def apply_filter_median(signal, filt):
    return np.convolve(signal, filt, mode='valid')

def impulse_response_median(filt):
    f = np.pad(filt, (0, len(filt) * 4), 'constant', constant_values=(0,0))
    return np.fft.fft(f)[:int(len(f) / 2)]


In [4]:
N = 500
kernel_size = 71
T = 1.0 # s

t = np.linspace(0, T, N)
s = np.random.rand(N)

filt = create_filter_median(kernel_size)
fs = apply_filter_median(s, filt)
ir = impulse_response_median(filt)

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)
tr_fs = go.Scatter(
    x = t,
    y = fs,
    name='filtered'
)

tr_ir = go.Scatter(
    x = [i for i in range(len(ir.real))],
    y = np.absolute(ir),
    name='impulse response'
)


fig = py.tools.make_subplots(rows = 2, cols = 1)

fig['layout'].update(height=900, width=900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)

fig.append_trace(tr_ir, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [4]:
def prod(iterable):
    from functools import reduce
    from operator import mul
    return reduce(mul, iterable, 1)

def transfer_function_butterworth(Rp, Rs, w0, w1):
    ep = np.sqrt(np.power(10, Rp / 10.) - 1)
    es = np.sqrt(np.power(10, Rs / 10.) - 1)
    k = w0 / w1
    k1 = ep / es
    N = int(np.ceil(np.log(k1) / np.log(k)))
    L = N // 2
    r = N % 2
    a = np.power(ep, - 1. / N)
    th = [(2 * n + 1) / 2 / N * np.pi for n in range(L)]
    
    return lambda s: 1. / (ep * np.power(s + a, r) * prod(s*s + 2*a*np.sin(thn)*s + a*a for thn in th))

In [125]:
N = 200
T = 2# s

t = np.linspace(0, T, N)
s = np.ones(N)

df = 1. / T
k = [i * df for i in range(N)]

Rp = 1.0
Rs = 30.0
w0 = 1
w1 = 5
HGen = transfer_function_butterworth(Rp, Rs, w0, w1)

H_jw = [HGen(1j*i / w0) for i in k]
afr = np.absolute(H_jw)
phase = np.angle(H_jw)

# sym = np.append(afr[::-1], afr)
coef = np.fft.ifft(afr)
# coef = [HGen((1 - 1. / z) / (1 + 1. / z) * 2 * N/T) for z in t[1:]]
# coef = coef[:len(coef) // 2]


coef[50:-50] = 0
H_jw = np.fft.fft(coef.real)
afr = np.absolute(H_jw)
afr = afr[:len(afr) // 2]


#--- 
kernel_size = 31
filt = create_filter_median(kernel_size)
filt = np.pad(filt, (0, N - kernel_size), 'constant', constant_values=(0, 0))
# ir_median = impulse_response_median(filt)
ir_median = np.fft.fft(filt)[:len(filt) // 2]
#---


tr_afr = go.Scatter(
    x = k,
    y = afr,
    name='Amplitude Frequency Response'
)

tr_ir_median = go.Scatter(
    x = k,
    y = np.absolute(ir_median),
    name='ir median'
)
tr_coef_real = go.Scatter(
    x = t,
    y = coef.real,
    name='coef real'
)

tr_coef_imag = go.Scatter(
    x = t,
    y = coef.imag,
    name='coef imag'
)

rows = 1
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)
# fig['layout'].update(xaxis=dict(type='log', autorange=True),
#                      xaxis2=dict(type='log', autorange=True),
#                      xaxis3=dict(type='log', autorange=True))

fig.append_trace(tr_afr, 1, 1)
fig.append_trace(tr_ir_median, 1, 1)
# fig.append_trace(tr_phase, 2, 1)
# fig.append_trace(tr_coef_real, 2, 1)
# fig.append_trace(tr_coef_imag, 2, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [109]:
f1 = 1
f2 = 8
w1 = 2 * np.pi * f1
w2 = 2 * np.pi * f2

s = [np.sin(w1 * i) + np.sin(w2 * i) for i in t]

filtsym = np.append(afr, afr[::-1])

filtered = np.fft.ifft(np.fft.fft(s) * filtsym).real

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)

tr_fs = go.Scatter(
    x = t,
    y = filtered,
    name='filtered'
)

rows = 1
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [123]:
f1 = 1
f2 = 8
w1 = 2 * np.pi * f1
w2 = 2 * np.pi * f2

s = [np.sin(w1 * i) + np.sin(w2 * i) for i in t]

filtsym = np.append(afr, afr[::-1])

filtered = np.fft.ifft(np.fft.fft(s) * filtsym).real

kernel_size = 31
filt = create_filter_median(kernel_size)
#filt = np.pad(filt, (0, N - kernel_size), 'constant', constant_values=(0, 0))
filtered_median = np.append(np.zeros(kernel_size // 2), np.convolve(s, filt, mode = 'valid'))

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)

tr_fs = go.Scatter(
    x = t,
    y = filtered,
    name='filtered'
)

tr_median = go.Scatter(
    x = t,
    y = filtered_median,
    name='filtered median'
)

rows = 1
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)
fig.append_trace(tr_median, 1, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [124]:
f = 1
w = 2 * np.pi * f

count = 4
s = [sum(np.cos(w * i * 2*k) for k in range(count)) for i in t]

filtsym = np.append(afr, afr[::-1])

filtered = np.fft.ifft(np.fft.fft(s) * filtsym).real

kernel_size = 31
filt = create_filter_median(kernel_size)
#filt = np.pad(filt, (0, N - kernel_size), 'constant', constant_values=(0, 0))
filtered_median = np.append(np.zeros(kernel_size // 2), np.convolve(s, filt, mode = 'valid'))


tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)

tr_fs = go.Scatter(
    x = t,
    y = filtered,
    name='filtered'
)

tr_median = go.Scatter(
    x = t,
    y = filtered_median,
    name='filtered median'
)


rows = 1
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)
fig.append_trace(tr_median, 1, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [28]:
def transfer_function_cheb(Rp, Rs, w0, w1):
    ep = np.sqrt(np.power(10, Rp / 10.) - 1)
    es = np.sqrt(np.power(10, Rs / 10.) - 1)
    k = w0 / w1
    k1 = ep / es
    N = int(np.ceil(np.arccosh(1. / k1) / np.arccosh(1. / k)))
    L = N // 2
    r = N % 2
    
    Gp = 1. / np.sqrt(1 + ep*ep)
    b = np.arcsinh(1. / ep) / N
    sigma0 = -np.sinh(b)
    a = [(2 * n + 1) / 2 / N * np.pi for n in range(L)]
    sigma = [-np.sin(a[k]) * np.sinh(b) for k in range(L)]
    w = [np.cos(a[k]) * np.cosh(b) for k in range(L)]
    
    abssquare = [sigma[k]*sigma[k] + w[k]*w[k] for k in range(L)]
    return lambda s: np.power(Gp, 1 - r) * np.power(-sigma0, r) * prod(abssquare) / np.power(s - sigma0, r) / prod(s*s - 2*sigma[k]*s + abssquare[k] for k in range(L))

In [29]:
N = 500
T = 10.0 # s

t = np.linspace(0, T, N)
s = np.ones(N)

df = 1. / T
k = [i * df for i in range(N)]

Rp = 1.0
Rs = 30.0
w0 = 1.0
w1 = 5.0
HGen = transfer_function_cheb(Rp, Rs, w0, w1)

H_jw = [HGen(1j*i / w0) for i in k]

afr = np.absolute(H_jw)
phase = [np.angle(HGen(1j*i)) for i in k]

sym = np.append(afr[::-1], afr)
ir = np.fft.ifft(sym)
ir = ir[:len(ir) // 2]

# tr_afr = go.Scatter(
#     x = np.linspace(-T, T, 2 * N - 1),
#     y = sym,
#     name='Amplitude Frequency Response'
# )

tr_afr = go.Scatter(
    x = k,
    y = afr,
    name='Amplitude Frequency Response'
)

tr_phase = go.Scatter(
    x = k,
    y = phase,
    name='Phase Frequency Response'
)

tr_ir = go.Scatter(
    x = k,
    y = np.absolute(ir),
    name='Impulse Response'
)

rows = 3
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)
fig['layout'].update(xaxis=dict(type='log', autorange=True),
                     xaxis2=dict(type='log', autorange=True),
                     xaxis3=dict(type='log', autorange=True))

fig.append_trace(tr_afr, 1, 1)
fig.append_trace(tr_phase, 2, 1)
fig.append_trace(tr_ir, 3, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]



In [30]:
N = 20
T = 1.0 # s

f1 = 1
f2 = 7
w1 = 2 * np.pi * f1
w2 = 2 * np.pi * f2

t = np.linspace(0, T, N)
s = [np.sin(w1 * i) + np.sin(w2 * i) for i in t]

df = 1. / T
k = [i * df for i in range(N)]

Rp = 1.0
Rs = 30.0
w0 = 1.0
w1 = 5.0
HGen = transfer_function_cheb(Rp, Rs, w0, w1)

H_jw = [HGen(1j*i / w0) for i in k]
afr = np.absolute(H_jw)

half = afr[:len(afr) // 2]
filtsym = np.append(half, half[::-1])

filtered = np.fft.ifft(np.fft.fft(s) * filtsym).real

tr_s = go.Scatter(
    x = t,
    y = s,
    name='signal'
)

tr_fs = go.Scatter(
    x = t,
    y = filtered,
    name='filtered'
)

rows = 1
cols = 1
fig = py.tools.make_subplots(rows = rows, cols = cols)

fig['layout'].update(height=rows * 450, width=cols * 900)

fig.append_trace(tr_s, 1, 1)
fig.append_trace(tr_fs, 1, 1)

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]

